In [3]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastai.vision.all import *
path = untar_data(URLs.PASCAL_2007)

In [4]:
path.ls()

In [5]:
df = pd.read_csv(path/'train.csv')
df.head()

In [6]:
df.iloc[:, 0]

In [7]:
df.iloc[0, :]

In [8]:
df.iloc[0]

In [9]:
df1 = pd.DataFrame()

In [10]:
df1["a"] = [1,2,3,4]
df1

In [11]:
df1['b'] = [10, 20, 30, 40]
df1['a'] + df1['b']

In [12]:
   dblock = DataBlock()

In [13]:
dsets = dblock.datasets(df)

In [14]:
dsets.train[0]

In [15]:
dblock = DataBlock(get_x = lambda r: r["fname"], get_y = lambda r: r["labels"])
dsets = dblock.datasets(df)
dsets.train[0]

In [16]:
def get_x(r): return path/"train"/r["fname"]
def get_y(r): return r['labels'].split(" ")
dblock = DataBlock(get_x = get_x, get_y = get_y)
dsets = dblock.datasets(df)
dsets.train[0]

In [17]:
dblock = DataBlock(blocks= (ImageBlock, MultiCategoryBlock) , get_x=get_x, 
                  get_y=get_y)
dsets = dblock.datasets(df)
dsets.train[0]

In [18]:
show_image(dsets.train[0][0])

In [19]:
idxs = torch.where(dsets.train[0][1]==1.)[0]

dsets.train.vocab[idxs]

In [20]:
def splitter(df):
    train = df.index[~df['is_valid']].tolist()
    valid = df.index[df['is_valid']].tolist()
    return train,valid

dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock), splitter=splitter, 
                  get_x=get_x, get_y=get_y)
dsets = dblock.datasets(df)
dsets.train[0]

In [21]:
dsets.valid[0]

In [22]:
dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock), splitter=splitter,
                  get_x=get_x, get_y=get_y,
                  item_tfms=RandomResizedCrop(128,min_scale=0.35))
dls = dblock.dataloaders(df)

In [23]:
dls.show_batch(nrows=1, ncols=3)

In [24]:
learn = cnn_learner(dls, resnet18)

In [28]:
x,y = dls.train.one_batch()
activs = learn.model(x)
activs.shape

In [29]:
activs[0]

In [30]:
loss_func = nn.BCEWithLogitsLoss()
loss = loss_func(activs, y)
loss

In [31]:
def accuracy_multi(inp, targ, thresh=0.5, sigmoid=True):
    if sigmoid: inp = inp.sigmoid()
    return ((inp>thresh)==targ.bool()).float().mean()

In [32]:
learn = cnn_learner(dls, resnet50, metrics=partial(accuracy_multi, thresh=0.2))
learn.fine_tune(3, base_lr=3e-3, freeze_epochs=4)

In [33]:
preds, targs = learn.get_preds()

In [35]:
xs = torch.linspace(0.05,0.95,29)
accs = [accuracy_multi(preds, targs, thresh=i, sigmoid=False) for i in xs]
plt.plot(xs,accs);